<a href="https://colab.research.google.com/github/Sibusisongwenya/WIP-Project/blob/main/Matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import sys
# Append the project root to the Python path
sys.path.append('/content/drive/MyDrive/uc')
!pip install torchbnn
os.chdir("/content/drive/MyDrive/uc")
print("Current working directory:", os.getcwd())

Current working directory: /content/drive/MyDrive/uc


In [6]:

import time
import random
import logging
import argparse

import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import transforms, models

# Import modules from our project
from dataset.ucmayo4 import UCMayo4
from utils.discretize import (discretize_to_four_class, discretize_binary, entropy_confidence, ci_confidence,
                              uncertainty_degree, probability_based_confidence)
from utils.magic import BayesianDenseNet121_LLSVI, DenseNet121_LLDropout, load_bayesian_model, load_mc_dropout_model
from utils.evaluation import evaluate_deterministic_model, evaluate_bayesian_model, calculate_uce
from utils.reporting import generate_clinical_report, generate_patient_case_studies
from utils.plotting import (
    plot_regression_uncertainty,
    plot_calibration_histograms,
    plot_uncertainty_histogram,
    plot_uncertainty_vs_error,
    plot_confusion_matrix,
    plot_reliability_diagram
)

# Configure logging (log to console and file "my_logs.log")
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler(), logging.FileHandler("my_logs.log")]
)
logging.info("Test log: Logging is configured!")

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
RANDOM_SEED = 35
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

def create_transform(resize: int, normalize: bool, augment: bool):
    transform_list = [transforms.Resize((resize, resize))]
    if augment:
        transform_list.extend([transforms.RandomHorizontalFlip(), transforms.RandomRotation(20)])
    transform_list.append(transforms.ToTensor())
    if normalize:
        transform_list.append(transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                    std=[0.229, 0.224, 0.225]))
    return transforms.Compose(transform_list)

def parse_arguments():
    parser = argparse.ArgumentParser(description="Mayo Score Prediction with Uncertainty")
    parser.add_argument("--train_dir", type=str, required=True, help="Path to training data")
    parser.add_argument("--calibration_dir", type=str, required=True, help="Path to calibration data")
    parser.add_argument("--test_dir", type=str, required=True, help="Path to test data")
    parser.add_argument("--checkpoint", type=str, required=True, help="Path to model checkpoint")
    parser.add_argument("--dropout_rate", type=float, default=0.3, help="Dropout rate")
    parser.add_argument("--num_mc_samples", type=int, default=50, help="Number of Monte Carlo samples")
    parser.add_argument("--batch_size", type=int, default=32, help="Batch size")

    # For Colab, override command-line arguments:
    if 'ipykernel' in sys.modules:
        args = parser.parse_args([
            "--train_dir", "/content/drive/MyDrive/uc/test_set/train",
            "--calibration_dir", "/content/drive/MyDrive/uc/test_set/calibration",
            "--test_dir", "/content/drive/MyDrive/uc/test_set/test",
            "--checkpoint", "/content/drive/MyDrive/uc/weights/best_DenseNet121.pth.tar",
            "--dropout_rate", "0.2",
            "--num_mc_samples", "50",
            "--batch_size", "64"
        ])
    else:
        args = parser.parse_args()
    return args

def load_dataloaders(args):
    train_transform = create_transform(256, True, True)
    val_transform = create_transform(256, True, False)
    test_transform = val_transform
    train_dataset = UCMayo4(root_dir=args.train_dir, transform=train_transform)
    val_dataset = UCMayo4(root_dir=args.calibration_dir, transform=val_transform)
    test_dataset = UCMayo4(root_dir=args.test_dir, transform=test_transform)
    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)
    return train_loader, val_loader, test_loader

def main():
    args = parse_arguments()
    logging.info("Arguments parsed successfully.")
    logging.info(f"Current working directory: {os.getcwd()}")

    train_loader, val_loader, test_loader = load_dataloaders(args)
    logging.info(f"DataLoaders loaded. (Train: {len(train_loader)} batches, Val: {len(val_loader)} batches, Test: {len(test_loader)} batches)")

    # Instantiate and load models using functions from magic.py
    bayesian_model_instance = load_bayesian_model(args.checkpoint, DEVICE, pretrained=True)
    dropout_model_instance = load_mc_dropout_model(args.checkpoint, DEVICE, pretrained=True, dropout_prob=args.dropout_rate)

    # Forward pass demonstration with dummy input (MC sampling enabled via sample=True)
    dummy_input = torch.randn(1, 3, 256, 256).to(DEVICE)
    bayesian_output = bayesian_model_instance(dummy_input, sample=True)
    dropout_output = dropout_model_instance(dummy_input, sample=True)
    logging.info(f"MC-Sampling output (Bayesian): {bayesian_output}")
    logging.info(f"MC-Sampling output (Dropout): {dropout_output}")

    # Evaluate models (example calls; further evaluation and reporting follow)
    deterministic_results = evaluate_deterministic_model(bayesian_model_instance, test_loader, DEVICE)
    bayesian_llsvi_results = evaluate_bayesian_model(
        bayesian_model_instance,
        test_loader,
        num_samples=args.num_mc_samples,
        probability_method='cdf',
        device=DEVICE,
        sub_batch_size=8
    )
    bayesian_lldropout_results = evaluate_bayesian_model(
        dropout_model_instance,
        test_loader,
        num_samples=args.num_mc_samples,
        probability_method='cdf',
        device=DEVICE,
        sub_batch_size=8
    )

    # Compute regression error metrics (continuous predictions vs. ground truth)
    det_cont_preds = deterministic_results['continuous_predictions']
    det_labels = deterministic_results['labels']
    det_mae = np.mean(np.abs(det_cont_preds - det_labels))
    det_rmse = np.sqrt(np.mean((det_cont_preds - det_labels) ** 2))

    bay_llsvi_cont_preds = bayesian_llsvi_results['continuous_predictions']
    bay_llsvi_mae = np.mean(np.abs(bay_llsvi_cont_preds - det_labels))
    bay_llsvi_rmse = np.sqrt(np.mean((bay_llsvi_cont_preds - det_labels) ** 2))

    bay_lldrop_cont_preds = bayesian_lldropout_results['continuous_predictions']
    bay_lldrop_mae = np.mean(np.abs(bay_lldrop_cont_preds - det_labels))
    bay_lldrop_rmse = np.sqrt(np.mean((bay_lldrop_cont_preds - det_labels) ** 2))

    # Compute binary classification metrics
    from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
    det_bin_preds = deterministic_results['binary_predictions']
    det_accuracy = accuracy_score(det_labels, det_bin_preds)
    det_f1 = f1_score(det_labels, det_bin_preds, average='weighted')
    try:
        det_auc = roc_auc_score(det_labels, det_bin_preds)
    except ValueError:  # Catch ValueError for single-class cases
        det_auc = 0  # Set to 0 when only one class is present

    bay_llsvi_bin_preds = bayesian_llsvi_results['binary_predictions']
    bay_llsvi_accuracy = accuracy_score(det_labels, bay_llsvi_bin_preds)
    bay_llsvi_f1 = f1_score(det_labels, bay_llsvi_bin_preds, average='weighted')
    try:
        bay_llsvi_auc = roc_auc_score(det_labels, bay_llsvi_bin_preds)
    except ValueError:  # Catch ValueError for single-class cases
        bay_llsvi_auc = 0  # Set to 0 when only one class is present

    bay_lldrop_bin_preds = bayesian_lldropout_results['binary_predictions']
    bay_lldrop_accuracy = accuracy_score(det_labels, bay_lldrop_bin_preds)
    bay_lldrop_f1 = f1_score(det_labels, bay_lldrop_bin_preds, average='weighted')
    try:
        bay_lldrop_auc = roc_auc_score(det_labels, bay_lldrop_bin_preds)
    except ValueError:  # Catch ValueError for single-class cases
        bay_lldrop_auc = 0  # Set to 0 when only one class is present


    # Compute Confidence/Uncertainty Metrics using computed values
    det_probs = np.array(deterministic_results.get('probabilities', np.eye(4)[det_bin_preds]))
    det_prob_conf = probability_based_confidence(det_probs)
    det_entropy_conf = np.mean(entropy_confidence(det_probs))
    det_ci_conf = np.mean(ci_confidence(np.array(deterministic_results['uncertainties'])))
    det_uc = np.mean(uncertainty_degree(det_probs))
    det_uce, _ = calculate_uce(det_labels, det_cont_preds, deterministic_results['uncertainties'])

    bay_llsvi_probs = np.array(bayesian_llsvi_results.get('cdf_probs'))
    bay_llsvi_prob_conf = probability_based_confidence(bay_llsvi_probs)
    bay_llsvi_entropy_conf = np.mean(entropy_confidence(bay_llsvi_probs))
    bay_llsvi_ci_conf = np.mean(ci_confidence(np.array(bayesian_llsvi_results['uncertainties'])))
    bay_llsvi_uc = np.mean(uncertainty_degree(bay_llsvi_probs))
    bay_llsvi_uce, _ = calculate_uce(bayesian_llsvi_results['labels'], bay_llsvi_cont_preds, bayesian_llsvi_results['uncertainties'])

    bay_lldrop_probs = np.array(bayesian_lldropout_results.get('cdf_probs'))
    bay_lldrop_prob_conf = probability_based_confidence(bay_lldrop_probs)
    bay_lldrop_entropy_conf = np.mean(entropy_confidence(bay_lldrop_probs))
    bay_lldrop_ci_conf = np.mean(ci_confidence(np.array(bayesian_lldropout_results['uncertainties'])))
    bay_lldrop_uc = np.mean(uncertainty_degree(bay_lldrop_probs))
    bay_lldrop_uce, _ = calculate_uce(bayesian_lldropout_results['labels'], bay_lldrop_cont_preds, bayesian_lldropout_results['uncertainties'])

    # Build overall metrics dictionary
    overall_metrics = {
        "Deterministic": {
            "accuracy": det_accuracy,
            "auc": det_auc,
            "binary_accuracy": det_accuracy,
            "f1": det_f1,
            "mae": det_mae,
            "rmse": det_rmse,
            "prob_conf": det_prob_conf,
            "entropy_conf": det_entropy_conf,
            "ci_conf": det_ci_conf,
            "uc": det_uc,
            "uce": det_uce
        },
        "Bayesian LL-SVI": {
            "accuracy": bay_llsvi_accuracy,
            "auc": bay_llsvi_auc,
            "binary_accuracy": bay_llsvi_accuracy,
            "f1": bay_llsvi_f1,
            "mae": bay_llsvi_mae,
            "rmse": bay_llsvi_rmse,
            "prob_conf": bay_llsvi_prob_conf,
            "entropy_conf": bay_llsvi_entropy_conf,
            "ci_conf": bay_llsvi_ci_conf,
            "uc": bay_llsvi_uc,
            "uce": bay_llsvi_uce
        },
        "Bayesian LL-Dropout": {
            "accuracy": bay_lldrop_accuracy,
            "auc": bay_lldrop_auc,
            "binary_accuracy": bay_lldrop_accuracy,
            "f1": bay_lldrop_f1,
            "mae": bay_lldrop_mae,
            "rmse": bay_lldrop_rmse,
            "prob_conf": bay_lldrop_prob_conf,
            "entropy_conf": bay_lldrop_entropy_conf,
            "ci_conf": bay_lldrop_ci_conf,
            "uc": bay_lldrop_uc,
            "uce": bay_lldrop_uce
        }
    }

    # Build a patient-level summary for a selected patient from Bayesian LL-SVI results.
    patient_idx = 0  # For example, choose the first sample
    patient_cont_pred = bayesian_llsvi_results['continuous_predictions'][patient_idx]
    patient_std = bayesian_llsvi_results['uncertainties'][patient_idx]
    patient_ground_truth = det_labels[patient_idx]
    reg_error = abs(patient_cont_pred - patient_ground_truth)
    patient_cdf_probs = bayesian_llsvi_results['cdf_probs'][patient_idx]
    patient_four_class_label = bayesian_llsvi_results['four_class_predictions'][patient_idx]
    patient_binary_label = bayesian_llsvi_results['binary_predictions'][patient_idx]
    patient_prob_conf = np.max(patient_cdf_probs)
    patient_entropy_conf = np.mean(entropy_confidence(np.expand_dims(patient_cdf_probs, axis=0)))
    patient_ci_conf = np.mean(ci_confidence(np.array([patient_std])))
    patient_uc = np.mean(uncertainty_degree(np.expand_dims(patient_cdf_probs, axis=0)))
    patient_uce, _ = calculate_uce(np.array([patient_ground_truth]), np.array([patient_cont_pred]), np.array([patient_std]))

    patient_summary = {
        "patient_id": "Patient 042",
        "predicted_score": patient_cont_pred,
        "std": patient_std,
        "ground_truth": patient_ground_truth,
        "reg_error": reg_error,
        "cdf_probs": patient_cdf_probs.tolist() if isinstance(patient_cdf_probs, np.ndarray) else patient_cdf_probs,
        "four_class_label": patient_four_class_label,
        "binary_label": patient_binary_label,
        "prob_conf": patient_prob_conf,
        "entropy_conf": patient_entropy_conf,
        "ci_conf": patient_ci_conf,
        "uc": patient_uc,
        "uce": patient_uce
    }

    from utils.reporting import generate_clinical_report
    generate_clinical_report(overall_metrics, patient_summary, output_dir='output', filename='clinical_report.txt')
    logging.info("Clinical report generated and saved to output/clinical_report.txt")

    from utils.plotting import plot_regression_uncertainty
    if bayesian_llsvi_results.get('continuous_predictions') is not None and bayesian_llsvi_results.get('uncertainties') is not None:
        plot_regression_uncertainty(
            np.array(bayesian_llsvi_results['continuous_predictions']),
            np.array(bayesian_llsvi_results['uncertainties']),
            np.array(det_labels)
        )
    logging.info("Pipeline execution completed.")


if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 196MB/s]


Missing keys: []
Unexpected keys: []
Missing keys: []
Unexpected keys: []
